In [1]:
!pip install paddle2onnx onnxruntime onnxslim onnxruntime-tools onnx  -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://pypi.org/simple/
  Using cached onnxslim-0.1.35-py3-none-any.whl.metadata (3.0 kB)
  Using cached onnxruntime_tools-1.7.0-py3-none-any.whl.metadata (14 kB)
  Using cached onnx-1.17.0-cp310-cp310-macosx_12_0_universal2.whl.metadata (16 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached py3nvml-0.2.7-py3-none-any.whl.metadata (13 kB)
  Using cached xmltodict-0.14.1-py2.py3-none-any.whl.metadata (8.0 kB)
Using cached onnxslim-0.1.35-py3-none-any.whl (140 kB)
Using cached onnxruntime_tools-1.7.0-py3-none-any.whl (212 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 2.3 MB/s eta 0:00:00a 0:00:01
Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)
Using cached py3nvml-0.2.7-py3-none-any.whl (55 kB)


In [1]:
!paddle2onnx --model_dir rapid_table_det_paddle/models/obj_det --model_filename model.pdmodel --params_filename model.pdiparams --save_file rapid_table_det/models/obj_det.onnx --opset_version 16 --enable_onnx_checker True
!paddle2onnx --model_dir rapid_table_det_paddle/models/db_net --model_filename model.pdmodel --params_filename model.pdiparams --save_file rapid_table_det/models/edge_det.onnx --opset_version 16 --enable_onnx_checker True
!paddle2onnx --model_dir rapid_table_det_paddle/models/pplcnet --model_filename model.pdmodel --params_filename model.pdiparams --save_file rapid_table_det/models/cls_det.onnx --opset_version 16 --enable_onnx_checker True

!onnxslim rapid_table_det/models/obj_det.onnx rapid_table_det/models/obj_det.onnx
!onnxslim rapid_table_det/models/edge_det.onnx rapid_table_det/models/edge_det.onnx
!onnxslim rapid_table_det/models/cls_det.onnx rapid_table_det/models/cls_det.onnx

[Paddle2ONNX] Start to parse PaddlePaddle model...
[Paddle2ONNX] Model file path: rapid_table_det_paddle/models/obj_det/model.pdmodel
[Paddle2ONNX] Paramters file path: rapid_table_det_paddle/models/obj_det/model.pdiparams
[Paddle2ONNX] Start to parsing Paddle model...
[Paddle2ONNX] Use opset_version = 16 for ONNX export.
[WARN][Paddle2ONNX] [multiclass_nms3: multiclass_nms3_0.tmp_1] [WARNING] Due to the operator multiclass_nms3, the exported ONNX model will only supports inference with input batch_size == 1.
[Paddle2ONNX] PaddlePaddle model is exported as ONNX format now.
2024-10-15 14:58:28 [INFO]	===============Make PaddlePaddle Better!================
2024-10-15 14:58:28 [INFO]	A little survey: https://iwenjuan.baidu.com/?code=r8hu2s
[Paddle2ONNX] Start to parse PaddlePaddle model...
[Paddle2ONNX] Model file path: rapid_table_det_paddle/models/db_net/model.pdmodel
[Paddle2ONNX] Paramters file path: rapid_table_det_paddle/models/db_net/model.pdiparams
[Paddle2ONNX] Start to parsing 

In [9]:
from pathlib import Path
import onnx
from onnxruntime.quantization import quantize_dynamic, QuantType, quant_pre_process
def quantize_model(root_dir_str, model_dir, pre_fix):

    original_model_path = f"{pre_fix}.onnx"
    # quantized_model_path = f"{pre_fix}_quantized.onnx"
    quantized_model_path = original_model_path
    original_model_path = f"{root_dir_str}/{model_dir}/{original_model_path}"
    quantized_model_path = f"{root_dir_str}/{model_dir}/{quantized_model_path}"
    quant_pre_process(original_model_path, quantized_model_path, auto_merge=True)
    # 进行动态量化
    quantize_dynamic(
        model_input=quantized_model_path,
        model_output=quantized_model_path,
        weight_type=QuantType.QUInt8
    )

    # 验证量化后的模型
    quantized_model = onnx.load(quantized_model_path)
    onnx.checker.check_model(quantized_model)
    print("Quantized model is valid.")

In [10]:
root_dir_str = "."
model_dir = f"rapid_table_det/models"
quantize_model(root_dir_str, model_dir,  "obj_det")
quantize_model(root_dir_str, model_dir,  "edge_det")
# quantize_model(root_dir_str, model_dir,  "cls_det")

Quantized model is valid.
Quantized model is valid.
